You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

### Import

In [1]:
import numpy as np
import re
from collections import Counter

import os
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import math 
plt.style.use('ggplot')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%matplotlib inline

In [4]:
def create_database(path, fmt='.csv'):
    fnames = [file for file in os.listdir(path) if file.endswith(fmt)]
    db = {}
    
    for file in fnames:
        table = file.split('.')[0]
        
        try:
            db[table] = pd.read_csv(path + '/' + file, encoding='latin-1')
        except:
            db[table] = pd.read_csv(path + '/' + file, lineterminator='\r', encoding='latin-1', error_bad_lines=False, delimiter=';')
        
    return db
            

In [5]:
database = create_database('/content/drive/MyDrive/FOCS/Datasets')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [6]:
database.keys()

dict_keys(['Books', 'Book-Ratings', 'Users', 'goodbooks', 'goodbooks-ratings'])

### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.
* OK

In [26]:
users = database['Users']

In [27]:
users

,User-ID,Location,Age,City,Region,Country
0,"\n""1""","nyc, new york, usa",NaN,nyc,new york,usa
1,"\n""2""","stockton, california, usa",18.0,stockton,california,usa
2,"\n""3""","moscow, yukon territory, russia",NaN,moscow,yukon territory,russia
3,"\n""4""","porto, v.n.gaia, portugal",17.0,porto,v.n.gaia,portugal
4,"\n""5""","farnborough, hants, united kingdom",NaN,farnborough,hants,united kingdom
...,...,...,...,...,...,...
278854,"\n""278855""","tacoma, washington, united kingdom",50.0,tacoma,washington,united kingdom
278855,"\n""278856""","brampton, ontario, canada",NaN,brampton,ontario,canada
278856,"\n""278857""","knoxville, tennessee, usa",NaN,knoxville,tennessee,usa
278857,"\n""278858""","dublin, n/a, ireland",NaN,dublin,n/a,ireland


In [28]:
users['City'] = users.Location.str.split(',', expand=True)[0]
users['Region'] = users.Location.str.split(',', expand=True)[1]
users['Country'] = users.Location.str.split(',', expand=True)[2]

In [29]:
users = users.replace('\n','', regex=True)

In [30]:
users = users.replace('"','', regex=True)

In [31]:
users

,User-ID,Location,Age,City,Region,Country
0,1,"nyc, new york, usa",NaN,nyc,new york,usa
1,2,"stockton, california, usa",18.0,stockton,california,usa
2,3,"moscow, yukon territory, russia",NaN,moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17.0,porto,v.n.gaia,portugal
4,5,"farnborough, hants, united kingdom",NaN,farnborough,hants,united kingdom
...,...,...,...,...,...,...
278854,278855,"tacoma, washington, united kingdom",50.0,tacoma,washington,united kingdom
278855,278856,"brampton, ontario, canada",NaN,brampton,ontario,canada
278856,278857,"knoxville, tennessee, usa",NaN,knoxville,tennessee,usa
278857,278858,"dublin, n/a, ireland",NaN,dublin,n/a,ireland


### 2. For each book in the *Books* dataset, compute its average rating.
* OK

In [48]:
books = database['Books']

In [49]:
books = books.replace('"','', regex=True)

In [50]:
books = books.replace('\n','', regex=True)

In [51]:
book_ratings = database['Book-Ratings']

In [52]:
book_ratings

,User-ID,ISBN,Book-Rating
0,"\n""276725""",034545104X,0.0
1,"\n""276726""",0155061224,5.0
2,"\n""276727""",0446520802,0.0
3,"\n""276729""",052165615X,3.0
4,"\n""276729""",0521795028,6.0
...,...,...,...
1149776,"\n""276706""",0679447156,0.0
1149777,"\n""276709""",0515107662,10.0
1149778,"\n""276721""",0590442449,10.0
1149779,"\n""276723""",05162443314,8.0


In [53]:
book_ratings_avg = book_ratings.groupby('ISBN', as_index=False).agg({'Book-Rating':'mean'})


In [54]:
book_ratings_avg

,ISBN,Book-Rating
0,0330299891,3.0
1,0375404120,1.5
2,0586045007,0.0
3,9022906116,3.5
4,9032803328,0.0
...,...,...
340551,cn113107,0.0
340552,ooo7156103,7.0
340553,Â§423350229,0.0
340554,Â´3499128624,8.0


In [55]:
books_w_ratings = books.join(book_ratings_avg.set_index('ISBN'), on='ISBN')

In [56]:
books_w_ratings

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,0.000000
...,...,...,...,...,...,...,...,...,...
271355,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,4.000000
271356,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,0.000000
271357,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,0.000000
271358,0767409752,A Guided Tour of Rene Descartes' Meditations o...,Christopher Biffle,2000,McGraw-Hill Humanities/Social Sciences/Languages,http://images.amazon.com/images/P/0767409752.0...,http://images.amazon.com/images/P/0767409752.0...,http://images.amazon.com/images/P/0767409752.0...,0.000000


### 3. For each book in the *GoodBooks* dataset, compute its average rating.
* OK

In [57]:
goodbooks = database['goodbooks']

In [58]:
goodbook_ratings = database['goodbooks-ratings']

In [59]:
goodbook_ratings

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
94,4,184,4
95,4,495,4
96,4,350,3
97,4,492,4


In [60]:
goodbooks.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [61]:
goodbooks_ids=[]
for index, row in goodbooks.iterrows():
  goodbooks_ids.append(row.book_id)

In [62]:
matchescount=0
for index, row in goodbook_ratings.iterrows():
  if row.book_id in goodbooks_ids:
    matchescount+=1
print('len: '+str(len(goodbook_ratings)))
print('matches: '+str(matchescount))

len: 99
matches: 30


The information 'book_id' contained in the 'goodbooks_ratings' dataset is not referring to the 'book_id' column of the 'goodbooks' dataset, as it's proven in the previous code cell (we only have 30 matches between the two lists). I also checked the "goodreads_book_id" and the "best_book_id" but both had less than 10 matches with the ratings id.

If the two attributes would have been linked, the task should have been accomplished like this:

In [63]:
#for each line in goodbooks ratings if rating 1 (ratings_1 of book_id) +1
for index, row in goodbook_ratings.iterrows():
  if row['rating']==1:
    goodbooks.loc[goodbooks['book_id'] == row['book_id'], 'ratings_1'] += 1
  if row['rating']==2:
    goodbooks.loc[goodbooks['book_id'] == row['book_id'], 'ratings_2'] += 1
  if row['rating']==3:
    goodbooks.loc[goodbooks['book_id'] == row['book_id'], 'ratings_3'] += 1
  if row['rating']==4:
    goodbooks.loc[goodbooks['book_id'] == row['book_id'], 'ratings_4'] += 1
  if row['rating']==5:
    goodbooks.loc[goodbooks['book_id'] == row['book_id'], 'ratings_5'] += 1
  print(index, row)

0 user_id      1
book_id    258
rating       5
Name: 0, dtype: int64
1 user_id       2
book_id    4081
rating        4
Name: 1, dtype: int64
2 user_id      2
book_id    260
rating       5
Name: 2, dtype: int64
3 user_id       2
book_id    9296
rating        5
Name: 3, dtype: int64
4 user_id       2
book_id    2318
rating        3
Name: 4, dtype: int64
5 user_id     2
book_id    26
rating      4
Name: 5, dtype: int64
6 user_id      2
book_id    315
rating       3
Name: 6, dtype: int64
7 user_id     2
book_id    33
rating      4
Name: 7, dtype: int64
8 user_id      2
book_id    301
rating       5
Name: 8, dtype: int64
9 user_id       2
book_id    2686
rating        5
Name: 9, dtype: int64
10 user_id       2
book_id    3753
rating        5
Name: 10, dtype: int64
11 user_id       2
book_id    8519
rating        5
Name: 11, dtype: int64
12 user_id     4
book_id    70
rating      4
Name: 12, dtype: int64
13 user_id      4
book_id    264
rating       3
Name: 13, dtype: int64
14 user_id      4

In [64]:
for index, row in goodbooks.iterrows():
  row['average_rating'] = (row['ratings_1']+row['ratings_2']*2+row['ratings_3']*3+row['ratings_4']*4+row['ratings_5']*5)/row['work_ratings_count']

### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.
* OK

In [ ]:
merged_books = books.loc[books.duplicated(subset=['Book-Title', 'Book-Author', 'Publisher'], keep=False), ['Book-Title', 'Book-Author', 'Publisher']]

In [ ]:
merged_books.drop_duplicates(subset=['Book-Title', 'Book-Author', 'Publisher'], keep='last', inplace=True)

In [ ]:
merged_books

,Book-Title,Book-Author,Publisher
749,The Rescue,Nicholas Sparks,Warner Books
886,The Door to December,Dean R. Koontz,Signet Book
1451,Skipping Christmas,JOHN GRISHAM,Doubleday
1565,Le Divorce (William Abrahams Book),Diane Johnson,Plume Books
1827,The Alchemist: A Fable About Following Your Dream,Paulo Coelho,HarperSanFrancisco
...,...,...,...
271142,Old Patagonian Express,Paul Theroux,Washington Square Press
271168,"The Two Towers (The Lord of the Rings, Part 2)",J. R. R. Tolkien,Houghton Mifflin
271171,R Is for Rocket,Ray Bradbury,Bantam Doubleday Dell
271222,Thirteen Days: A Memoir of the Cuban Missile C...,Robert F. Kennedy,Signet Book


### 5. For each book in `merged books` compute its average rating.
* OK

The average is computed considering all books in `books` that have been merged.

In [ ]:
books_w_ratings.head(1)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0.0


In [ ]:
booksrattemp=books_w_ratings[['ISBN','Book-Title','Book-Author','Publisher','Book-Rating']]

In [ ]:
booksrattemp

,ISBN,Book-Title,Book-Author,Publisher,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,Oxford University Press,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,HarperPerennial,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton &amp; Company,0.000000
...,...,...,...,...,...
271355,0525447644,From One to One Hundred,Teri Sloat,Dutton Books,4.000000
271356,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,HarperSanFrancisco,0.000000
271357,0192126040,Republic (World's Classics),Plato,Oxford University Press,0.000000
271358,0767409752,A Guided Tour of Rene Descartes' Meditations o...,Christopher Biffle,McGraw-Hill Humanities/Social Sciences/Languages,0.000000


In [ ]:
ratings = booksrattemp.groupby(['Book-Title','Book-Author','Publisher'], as_index=False)['Book-Rating'].agg('mean')

In [ ]:
ratings

,Book-Title,Book-Author,Publisher,Book-Rating
0,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,Hyperion Books for Children,2.250000
1,Always Have Popsicles,Rebecca Harvin,Rebecca L. Harvin,0.000000
2,Apple Magic (The Collector's series),Martina Boudreau,Amer Cooking Guild,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",Nancy N. Rue,Zonderkidz,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,Lou Mobley,"Teleonet, Incorporated",0.000000
...,...,...,...,...
266122,Ã?Ã?lpiraten.,Janwillem van de Wetering,Rowohlt Tb.,0.000000
266123,Ã?Ã?rger mit Produkt X. Roman.,Joan Aiken,Diogenes Verlag,5.250000
266124,Ã?Ã?sterlich leben.,Anselm GrÃ?ÃÂ¼n,"Herder, Freiburg",7.000000
266125,Ã?Ã?stlich der Berge.,David Guterson,btb,2.666667


In [ ]:
merged_books_w_ratings = pd.merge(merged_books,ratings,on=['Book-Title','Book-Author','Publisher'], how='left')

In [ ]:
merged_books_w_ratings.head()

,Book-Title,Book-Author,Publisher,Book-Rating
0,The Rescue,Nicholas Sparks,Warner Books,3.235165
1,The Door to December,Dean R. Koontz,Signet Book,2.650565
2,Skipping Christmas,JOHN GRISHAM,Doubleday,3.863814
3,Le Divorce (William Abrahams Book),Diane Johnson,Plume Books,2.663490
4,The Alchemist: A Fable About Following Your Dream,Paulo Coelho,HarperSanFrancisco,4.267544


### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

* OK

In [ ]:
merged_books.head(1)

,Book-Title,Book-Author,Publisher
749,The Rescue,Nicholas Sparks,Warner Books


In [ ]:
books_w_ratings.head(1)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0.0


In [ ]:
merged_books["min_avg_rating"] = ""
merged_books["max_avg_rating"] = ""

In [ ]:
for index, row in merged_books.iterrows():
  row['min_avg_rating'] = books_w_ratings[(books_w_ratings['Book-Title']==row['Book-Title']) & (books_w_ratings['Book-Author']==row['Book-Author']) & (books_w_ratings['Publisher']==row['Publisher'])]['Book-Rating'].min()
  row['max_avg_rating'] = books_w_ratings[(books_w_ratings['Book-Title']==row['Book-Title']) & (books_w_ratings['Book-Author']==row['Book-Author']) & (books_w_ratings['Publisher']==row['Publisher'])]['Book-Rating'].max()

In [ ]:
merged_books

,Book-Title,Book-Author,Publisher,min_avg_rating,max_avg_rating
749,The Rescue,Nicholas Sparks,Warner Books,3.08571,3.38462
886,The Door to December,Dean R. Koontz,Signet Book,2.48462,2.81651
1451,Skipping Christmas,JOHN GRISHAM,Doubleday,3.47525,4.25238
1565,Le Divorce (William Abrahams Book),Diane Johnson,Plume Books,2.64516,2.68182
1827,The Alchemist: A Fable About Following Your Dream,Paulo Coelho,HarperSanFrancisco,3.7193,4.81579
...,...,...,...,...,...
271142,Old Patagonian Express,Paul Theroux,Washington Square Press,0,8
271168,"The Two Towers (The Lord of the Rings, Part 2)",J. R. R. Tolkien,Houghton Mifflin,7.6,10
271171,R Is for Rocket,Ray Bradbury,Bantam Doubleday Dell,0,0
271222,Thirteen Days: A Memoir of the Cuban Missile C...,Robert F. Kennedy,Signet Book,0,0


### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

* OK

In [ ]:
list_auth=[]
for index, row in goodbooks.iterrows():
  splitted_auth = row['authors'].split(',')
  for auth in splitted_auth:
    if auth not in list_auth:
      list_auth.append(auth)
print(list_auth)


['Suzanne Collins', 'J.K. Rowling', ' Mary GrandPrÃ©', 'Stephenie Meyer', 'Harper Lee', 'F. Scott Fitzgerald', 'John Green', 'J.R.R. Tolkien', 'J.D. Salinger', 'Dan Brown', 'Jane Austen', 'Khaled Hosseini', 'Veronica Roth', 'George Orwell', ' Erich Fromm', ' CelÃ¢l Ã\x9cster', 'Anne Frank', ' Eleanor Roosevelt', ' B.M. Mooyaart-Doubleday', 'Stieg Larsson', ' Reg Keeland', ' Rufus Beck', 'Alice Sebold', 'William Golding', 'William Shakespeare', ' Robert           Jackson', 'Gillian Flynn', 'Kathryn Stockett', 'John Steinbeck', 'Arthur Golden', 'E.L. James', 'Paulo Coelho', ' Alan R. Clarke', 'Lois Lowry', 'C.S. Lewis', 'Audrey Niffenegger', 'George R.R. Martin', 'Elizabeth Gilbert', 'Rick Riordan', 'Louisa May Alcott', 'Charlotte BrontÃ«', ' Michael Mason', 'Nicholas Sparks', 'Yann Martel', 'Sara Gruen', 'Markus Zusak', 'Ray Bradbury', 'Shel Silverstein', 'Cassandra Clare', 'Christopher Paolini', 'Douglas Adams', 'Aldous Huxley', 'Sue Monk Kidd', 'Mark Twain', ' John Seelye', ' Guy Card

In [ ]:
df_authors = pd.DataFrame()
df_authors['author']=list_auth
df_authors['work_text_reviews_count']=0.0

In [ ]:
for index1, row1 in df_authors.iterrows():    
  for index, row in goodbooks.iterrows():
    if row1['author'] in row['authors']:
      num_authors=len(row['authors'].split(','))
      df_authors.loc[index1, 'work_text_reviews_count']+=(row['work_text_reviews_count']/num_authors)

In [ ]:
df_authors

,author,work_text_reviews_count
0,Suzanne Collins,340066.000000
1,J.K. Rowling,136668.000000
2,Mary GrandPrÃ©,136668.000000
3,Stephenie Meyer,258573.000000
4,Harper Lee,72586.000000
...,...,...
105,Oscar Wilde,9823.500000
106,Jeffrey Eugenides,9823.500000
107,Bram Stoker,5754.333333
108,Nina Auerbach,5754.333333


### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.
* OK

In [ ]:
for year in goodbooks['original_publication_year'].unique():
  list_auth=[]
  for index, row in goodbooks[goodbooks['original_publication_year']==year].iterrows():
    splitted_auth = row['authors'].split(',')
    for auth in splitted_auth:
      if auth not in list_auth:
        list_auth.append(auth)
  df_authors_year = pd.DataFrame()
  df_authors_year['author']=list_auth
  df_authors_year['work_text_reviews_sum']=0.0
  for index1, row1 in df_authors_year.iterrows():    
    for index, row in goodbooks[goodbooks['original_publication_year']==year].iterrows():
      if row1['author'] in row['authors']:
        num_authors=len(row['authors'].split(','))
        df_authors_year.loc[index1, 'work_text_reviews_sum']+=(row['work_text_reviews_count']/num_authors)
  print('For year '+str(year)+' most reviewed author is ')
  print(df_authors_year[df_authors_year['work_text_reviews_sum'] == df_authors_year['work_text_reviews_sum'].max()])
  del df_authors_year, splitted_auth, list_auth, num_authors

For year 2008.0 most reviewed author is 
            author  work_text_reviews_sum
0  Suzanne Collins               155254.0
For year 1997.0 most reviewed author is 
            author  work_text_reviews_sum
0     J.K. Rowling                37933.5
1   Mary GrandPrÃ©                37933.5
For year 2005.0 most reviewed author is 
            author  work_text_reviews_sum
0  Stephenie Meyer                95009.0
For year 1960.0 most reviewed author is 
       author  work_text_reviews_sum
0  Harper Lee                72586.0
For year 1925.0 most reviewed author is 
                author  work_text_reviews_sum
0  F. Scott Fitzgerald                51992.0
For year 2012.0 most reviewed author is 
       author  work_text_reviews_sum
0  John Green               140739.0
For year 1937.0 most reviewed author is 
           author  work_text_reviews_sum
0  J.R.R. Tolkien                37653.0
For year 1951.0 most reviewed author is 
          author  work_text_reviews_sum
0  J.D. Salinger

### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets
* OK

In [ ]:
common_books = pd.merge(books_w_ratings,goodbooks,left_on='ISBN', right_on='isbn', how='inner')

In [ ]:
common_books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1400032717,The Curious Incident of the Dog in the Night-T...,Mark Haddon,2004,Vintage,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,4.813187,60,1618,1618,4259809,183,1400032717,9.781400e+12,Mark Haddon,2003.0,The Curious Incident of the Dog in the Night-Time,The Curious Incident of the Dog in the Night-Time,eng,3.85,867553,915790,35796,21049,58117,218218,355925,262481,https://images.gr-assets.com/books/1479863624m...,https://images.gr-assets.com/books/1479863624s...
1,014028009X,Bridget Jones's Diary,Helen Fielding,1999,Penguin Books,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,3.751852,75,227443,227443,3185154,193,014028009X,9.780140e+12,Helen Fielding,1996.0,NaN,"Bridget Jones's Diary (Bridget Jones, #1)",eng,3.75,736498,754498,8157,28973,59309,198555,254836,212825,https://images.gr-assets.com/books/1292060045m...,https://images.gr-assets.com/books/1292060045s...
2,014038572X,The Outsiders (Now in Speak!),S. E. Hinton,1997,Puffin Books,http://images.amazon.com/images/P/014038572X.0...,http://images.amazon.com/images/P/014038572X.0...,http://images.amazon.com/images/P/014038572X.0...,4.460674,90,231804,231804,1426690,156,014038572X,9.780140e+12,S.E. Hinton,1967.0,The Outsiders,The Outsiders,en-US,4.06,659248,680437,22662,10178,34346,134244,227250,274419,https://images.gr-assets.com/books/1442129426m...,https://images.gr-assets.com/books/1442129426s...
3,1594480001,The Kite Runner,Khaled Hosseini,2004,Riverhead Books,http://images.amazon.com/images/P/1594480001.0...,http://images.amazon.com/images/P/1594480001.0...,http://images.amazon.com/images/P/1594480001.0...,2.400000,11,77203,77203,3295919,283,1594480001,9.781594e+12,Khaled Hosseini,2003.0,The Kite Runner,The Kite Runner,eng,4.26,1813044,1878095,59730,34288,59980,226062,628174,929591,https://images.gr-assets.com/books/1484565687m...,https://images.gr-assets.com/books/1484565687s...
4,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,2004,Scholastic Paperbacks,http://images.amazon.com/images/P/043965548X.0...,http://images.amazon.com/images/P/043965548X.0...,http://images.amazon.com/images/P/043965548X.0...,3.533333,18,5,5,2402163,376,043965548X,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck",1999.0,Harry Potter and the Prisoner of Azkaban,Harry Potter and the Prisoner of Azkaban (Harr...,eng,4.53,1832823,1969375,36099,6716,20413,166129,509447,1266671,https://images.gr-assets.com/books/1499277281m...,https://images.gr-assets.com/books/1499277281s...


In [ ]:
common_books['avg_rat_difference']=abs(common_books['average_rating']-common_books['Book-Rating'])

In [ ]:
common_books['avg_rat_difference']

0    0.963187
1    0.001852
2    0.400674
3    1.860000
4    0.996667
Name: avg_rat_difference, dtype: float64

### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.
* OK

In [40]:
users["Age-group"] = pd.cut(x=users['Age'], bins=[0,14,24,34,44,54,64,74,84,94,104,np.inf], labels=['0-14','15-24','25-34','35-44','45-54','55-64','65-74','75-84','85-94','95-104','105+'])

In [36]:
users

,User-ID,Location,Age,City,Region,Country,Age-group
0,1,"nyc, new york, usa",NaN,nyc,new york,usa,NaN
1,2,"stockton, california, usa",18.0,stockton,california,usa,15-24
2,3,"moscow, yukon territory, russia",NaN,moscow,yukon territory,russia,NaN
3,4,"porto, v.n.gaia, portugal",17.0,porto,v.n.gaia,portugal,15-24
4,5,"farnborough, hants, united kingdom",NaN,farnborough,hants,united kingdom,NaN
...,...,...,...,...,...,...,...
278854,278855,"tacoma, washington, united kingdom",50.0,tacoma,washington,united kingdom,45-54
278855,278856,"brampton, ontario, canada",NaN,brampton,ontario,canada,NaN
278856,278857,"knoxville, tennessee, usa",NaN,knoxville,tennessee,usa,NaN
278857,278858,"dublin, n/a, ireland",NaN,dublin,n/a,ireland,NaN


In [44]:
users_0_14=users[users['Age-group']=='0-14']
users_15_24=users[users['Age-group']=='15-24']
users_25_34=users[users['Age-group']=='25-34']
users_35_44=users[users['Age-group']=='35-44']
users_45_54=users[users['Age-group']=='45-54']
users_55_64=users[users['Age-group']=='55-64']
users_65_74=users[users['Age-group']=='65-74']
users_75_84=users[users['Age-group']=='75-84']
users_85_94=users[users['Age-group']=='85-94']
users_95_104=users[users['Age-group']=='95-104']
users_over105=users[users['Age-group']=='105+']
users_na=users[users['Age-group'].isna()]


In [47]:
len(users)==len(users_na)+len(users_0_14)+len(users_15_24)+len(users_25_34)+len(users_35_44)+len(users_45_54)+len(users_55_64)+len(users_65_74)+len(users_75_84)+len(users_85_94)+len(users_95_104)+len(users_over105)

True

### 11. Find the books that appear only in the goodbooks datasets.
* OK

In [69]:
goodbooksonly = goodbooks[~(goodbooks['isbn'].isin(books['ISBN'])) & goodbooks['original_title'].isin(books['Book-Title'])].reset_index(drop=True)


### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?
* OK

In [74]:
most_authortitle= pd.DataFrame(books.groupby(['Book-Author', 'Book-Title']).size(), columns=['count']).reset_index()

In [ ]:
most_authortitle=most_authortitle.iloc[most_authortitle['count'].idxmax()]

In [83]:
most_authortitle

Book-Author    Louisa May Alcott
Book-Title          Little Women
count                         21
Name: 150649, dtype: object

### 13. Find the author with the highest average rating according to the goodbooks datasets.
* OK

In [88]:
list_auth=[]
for index, row in goodbooks.iterrows():
  splitted_auth = row['authors'].split(',')
  for auth in splitted_auth:
    if auth not in list_auth:
      list_auth.append(auth)
df_authors = pd.DataFrame()
df_authors['author']=list_auth
df_authors['average_rating']=0.0
df_authors['ratings_count']=0
for index1, row1 in df_authors.iterrows():    
  for index, row in goodbooks.iterrows():
    if row1['author'] in row['authors']:
      df_authors.loc[index1, 'ratings_count']+=1
      df_authors.loc[index1, 'average_rating']+=row['average_rating']
df_authors['average_rating']=df_authors['average_rating']/df_authors['ratings_count']
print('The author with the highest avg rating is: ')
print(df_authors[df_authors['average_rating'] == df_authors['average_rating'].max()])

The author with the highest avg rating is: 
         author  average_rating  ratings_count
21   Rufus Beck            4.53              1
